<a href="https://colab.research.google.com/github/amirsakib16/Computer_Graphics/blob/main/Diamond_Catcher_with_AABBcollisionDetection_and_LineClipping_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import time

width, height = 800, 600
horizontalCatPos = 350 / 800
widthCat = 100 / 800
horizontalDiaPos = random.uniform(50/800, 700/800)
verticalDiaPos = 550 / 600
sizeDia = 20 / 800
score = 0
diamondFallSpeed = 3
isPaused = False
endGame = False
endGame_handled = False
prevTime = time.time()
diamondRGB = (
    random.uniform(0.8, 1.0),
    random.uniform(0.7, 1.0),
    random.uniform(0.9, 1.0) )


def Xscaled(ratio):
    return int(width * ratio)
def Yscaled(ratio):
    return int(height * ratio)

def triggerPixel(x, y):
    glBegin(GL_POINTS)
    glVertex2i(x, y)
    glEnd()

def cohen_S_algo(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    dxAbsolute = abs(dx)
    dyAbsolute = abs(dy)
    horizontal = {dx>=0 and dy>=0:0, dx>=0 and dy<0:7,
                    dx<0 and dy>=0:3, dx<0 and dy<0:4}
    vertical = {dx>=0 and dy>=0:1, dx>=0 and dy<0:6,
                    dx<0 and dy>=0:2, dx<0 and dy<0:5}
    if dxAbsolute >= dyAbsolute:
        return horizontal[True]
    else:
        return vertical[True]

def zoneNto0(x, y, zone):
    zoneConversion = {0:(x,y),1:(y,x),2:(y,-x),3:(-x, y),
                    4:(-x, -y),5:(-y, -x),6:(-y, x),7:(x, -y)}
    return zoneConversion[zone]

def zone0toN(x, y, zone):
    reverseConversion = {0:(x, y),1:(y, x),2:(-y, x),3:(-x, y),
                        4:(-x, -y),5:(-y, -x),6:(y, -x),7:(x, -y)}
    return reverseConversion[zone]

def lineDraw_Algo(x1, y1, x2, y2):
    zone = cohen_S_algo(x1, y1, x2, y2)
    x1E, y1E = zoneNto0(x1, y1, zone)
    x2E, y2E = zoneNto0(x2, y2, zone)
    dx = x2E - x1E
    dy = y2E - y1E
    d = 2 * dy - dx
    dE = 2 * dy
    dNE = 2*dy - 2*dx
    x, y = x1E, y1E
    while x <= x2E:
        px, py = zone0toN(x, y, zone)
        triggerPixel(int(px), int(py))
        if d > 0:
            y += 1
            d += dNE
        else:
            d += dE
        x += 1

def DIAMOND(x,y,size):
    glColor3f(*diamondRGB)
    lineDraw_Algo(x, y, x + size, y + size)
    lineDraw_Algo(x, y, x - size, y + size)
    lineDraw_Algo(x - size, y + size, x, y + (size + size))
    lineDraw_Algo(x + size, y + size, x, y + (size + size))

def CATCHER():
    thickness = 20
    x = Xscaled(horizontalCatPos)
    y = 0
    catcherWidth = Xscaled(widthCat)
    if not endGame:
        glColor3f(1, 1, 1)
    else:
        glColor3f(1, 0, 0)
    lineDraw_Algo(x + int(catcherWidth * 0.1), y,
                    x + catcherWidth - int(catcherWidth * 0.1), y)
    lineDraw_Algo(x, y + thickness,
                    x + int(catcherWidth * 0.1), y)
    lineDraw_Algo(x + catcherWidth - int(catcherWidth * 0.1), y,
                    x + catcherWidth, y + thickness)
    lineDraw_Algo(x, y + thickness,
                    x + catcherWidth, y + thickness)

def BUTTONS():
    glColor3f(0, 1, 1)
    lineDraw_Algo(Xscaled(50/800), Yscaled(570/600),
                    Xscaled(30/800), Yscaled(580/600))
    lineDraw_Algo(Xscaled(30/800), Yscaled(580/600),
                    Xscaled(50/800), Yscaled(590/600))
    lineDraw_Algo(Xscaled(30/800), Yscaled(580/600),
                    Xscaled(70/800), Yscaled(580/600))
    glColor3f(1, 0.75, 0)
    if not isPaused:
        lineDraw_Algo(Xscaled(390/800), Yscaled(570/600),
                        Xscaled(390/800), Yscaled(590/600))
        lineDraw_Algo(Xscaled(410/800), Yscaled(570/600),
                        Xscaled(410/800), Yscaled(590/600))
    else:
        lineDraw_Algo(Xscaled(390/800), Yscaled(570/600),
                        Xscaled(410/800), Yscaled(580/600))
        lineDraw_Algo(Xscaled(410/800), Yscaled(580/600),
                        Xscaled(390/800), Yscaled(590/600))
        lineDraw_Algo(Xscaled(390/800), Yscaled(590/600),
                        Xscaled(390/800), Yscaled(570/600))
    glColor3f(1, 0, 0)
    lineDraw_Algo(Xscaled(740/800), Yscaled(570/600),
                    Xscaled(760/800), Yscaled(590/600))
    lineDraw_Algo(Xscaled(760/800), Yscaled(570/600),
                    Xscaled(740/800), Yscaled(590/600))

def COLLISION_CHECKER():
    global endGame
    global score
    global verticalDiaPos
    global horizontalDiaPos
    global diamondFallSpeed
    global diamondRGB
    global sizeDia
    catcherYpos = 0
    catcherXpos = Xscaled(horizontalCatPos)
    catcherWidth = Xscaled(widthCat)
    catcherBoxArea = (catcherXpos, catcherYpos, catcherWidth, 20)
    diamondXpos = Xscaled(horizontalDiaPos)
    diamondYpos = Yscaled(verticalDiaPos)
    diamondSize = Xscaled(sizeDia)
    diamondBoxArea = (diamondXpos - diamondSize, diamondYpos, 2 * diamondSize, 2 * diamondSize)

    if (catcherBoxArea[0] < diamondBoxArea[0] + diamondBoxArea[2] and catcherBoxArea[0] + catcherBoxArea[2] > diamondBoxArea[0] and
        catcherBoxArea[1] < diamondBoxArea[1] + diamondBoxArea[3] and catcherBoxArea[1] + catcherBoxArea[3] > diamondBoxArea[1]):
        score += 1
        verticalDiaPos = 550 / 600
        horizontalDiaPos = random.uniform(50 / 800, 700 / 800)
        diamondFallSpeed += 0.40
        diamondRGB = (random.random(), random.random(), random.random())
        print("Score:", score)

def display():
    global verticalDiaPos
    global endGame
    global prevTime
    global endGame_handled
    global isPaused
    touchGround = Yscaled(verticalDiaPos) < 0
    glClear(GL_COLOR_BUFFER_BIT)
    BUTTONS()
    delta = time.time() - prevTime
    prevTime = time.time()
    if isPaused == False:
        if endGame == False:
            verticalDiaPos -= (diamondFallSpeed * delta * 60) / height
        if touchGround and not endGame_handled:
            endGame = True
            endGame_handled = True
            print(f"Game Over! Score: {score}")
    if not endGame or isPaused:
        DIAMOND(Xscaled(horizontalDiaPos),
                        Yscaled(verticalDiaPos),
                        Xscaled(sizeDia))
    CATCHER()
    COLLISION_CHECKER()
    glutSwapBuffers()

def dynamicWindow(new_width, new_height):
    global width
    global height
    global prevTime
    if new_height == 0:
        new_height = 1
    width = new_width
    height = new_height
    glViewport(0, 0, width, height)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    prevTime = time.time()

def KEYBOARD(key, x, y):
    global horizontalCatPos
    catcherWidth = Xscaled(widthCat)
    old_catcher_x = Xscaled(horizontalCatPos)
    if endGame or isPaused:
        return
    if key == GLUT_KEY_LEFT and old_catcher_x > 0:
        horizontalCatPos = max(0, (old_catcher_x - 10) / width)
    elif key == GLUT_KEY_RIGHT and old_catcher_x + catcherWidth < width:
        horizontalCatPos = min((width - catcherWidth) / width, (old_catcher_x + 10) / width)

def Mouse(button, state, x, y):
    global isPaused
    global endGame
    global horizontalCatPos
    global verticalDiaPos
    global horizontalDiaPos
    global score
    global diamondFallSpeed
    global diamondRGB
    global prevTime
    global endGame_handled

    if state == GLUT_DOWN:
        ogl_x = x
        ogl_y = height - y
        RESTART_BUTTON = Xscaled(30/800) <= ogl_x <= Xscaled(70/800) and Yscaled(570/600) <= ogl_y <= Yscaled(590/600)
        PLAY_AND_RESUME = Xscaled(390/800) <= ogl_x <= Xscaled(410/800) and Yscaled(570/600) <= ogl_y <= Yscaled(590/600)
        EXIT_BUTTON = Xscaled(740/800) <= ogl_x <= Xscaled(760/800) and Yscaled(570/600) <= ogl_y <= Yscaled(590/600)
        if RESTART_BUTTON:
            isPaused = False
            endGame = False
            endGame_handled = False
            score = 0
            verticalDiaPos = 550 / 600
            horizontalDiaPos = random.uniform(50 / 800, 700 / 800)
            diamondFallSpeed = 3
            diamondRGB = (random.random(), random.random(), random.random())
            prevTime = time.time()
            print("Starting over!")
        elif PLAY_AND_RESUME:
            isPaused = not isPaused
        elif EXIT_BUTTON:
            print("Goodbye! Score:", score)
            glutLeaveMainLoop()

def timer(v):
    glutPostRedisplay()
    glutTimerFunc(16, timer, 0)

def window():
    glClearColor(0, 0, 0, 1)
    gluOrtho2D(0, width, 0, height)

def main():
    glutInit()
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
    glutInitWindowSize(width, height)
    glutCreateWindow(b"Catch the Diamonds!")
    window()
    glutDisplayFunc(display)
    glutSpecialFunc(KEYBOARD)
    glutMouseFunc(Mouse)
    glutReshapeFunc(dynamicWindow)
    glutTimerFunc(0, timer, 0)
    glutMainLoop()

main()